In [14]:
import pandas as pd
import numpy as np

import pickle

from Constants import *

from scipy.sparse import csr_matrix, dok_matrix
from scipy import sparse

from sklearn.feature_extraction.text import TfidfTransformer

In [31]:
df_newspaper_speaker_count = pd.read_csv(FILE_NEWSPAPER_SPEAKER_COUNT, compression='bz2')
df_newspaper_speaker_count

,newspaper,speaker,count
0,1011now,adam morfeld,4
1,1011now,adam schiff,2
2,1011now,adrian smith,2
3,1011now,alexandra brown,1
4,1011now,alfonso morales,1
...,...,...,...
2434405,zigwheels,srinivas reddy,1
2434406,zigwheels,toby price,1
2434407,zigwheels,yash aradhya,1
2434408,zip06,mike caruso,16


### Create newspaper set and indexes
We create the sorted set of newspapesr and the indexes to link the matrix entries to newspapers.

In [32]:
newspapers = set(df_newspaper_speaker_count.newspaper)
newspapers = sorted(list(newspapers))
newspaper_to_index = {s:i for i,s in enumerate(newspapers)}
index_to_newspaper = {i:s for i,s in enumerate(newspapers)}

In [33]:
with open(PICKLE_NEWSPAPER_TO_INDEX, 'wb') as handle:
    pickle.dump(newspaper_to_index, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [34]:
with open(PICKLE_INDEX_TO_NEWSPAPER, 'wb') as handle:
    pickle.dump(index_to_newspaper, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Create speaker set and indexes
We create the sorted set of speakers and the indexes to link the matrix entries to speakers.

In [42]:
speakers = set(df_newspaper_speaker_count.speaker)
speakers.remove("none")
speakers = sorted(list(speakers))

speaker_to_index = {s:i for i,s in enumerate(speakers)}
index_to_speaker = {i:s for i,s in enumerate(speakers)}

    
with open(PICKLE_SPEAKER_TO_INDEX, 'wb') as handle:
    pickle.dump(speaker_to_index, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(PICKLE_INDEX_TO_SPEAKER, 'wb') as handle:
    pickle.dump(index_to_speaker, handle, protocol=pickle.HIGHEST_PROTOCOL)


212146
212145
[' hacksaw  jim duggan', ' hang', ' lane', ' mad mike ', ' palmer', ' rich', ' rowdy  roddy piper', ' sandy  alex g', ' superstar  billy graham', ' the fox ']


### Create frequency matrix
We generate a frequency matrix #newspaper x #speaker: entry (i,j) is the number of time speaker j appears in newspaper i.

In [37]:
newspaper_speaker_frequency_matrix = dok_matrix((len(newspapers), len(speakers)), dtype=np.uint32)

df_newspaper_speaker_count_not_none = df_newspaper_speaker_count[df_newspaper_speaker_count['speaker']!='none']
for index, row in df_newspaper_speaker_count_not_none.iterrows():
    if index % 10000==0:
        print(index)
    index_speaker = speaker_to_index[row['speaker']]
    index_newspaper = newspaper_to_index[row['newspaper']]
    newspaper_speaker_frequency_matrix[index_newspaper, index_speaker] = row["count"]
    
newspaper_speaker_frequency_csr = newspaper_speaker_frequency_matrix.tocsr()

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
13

### Create TF-IDF matrix
Transform the frequency matrix into a TF-IDF matrix. Each row is normalised and each column is scaled by proportionnaly to the number of newspaper in which the speaker is quoted.

In [38]:
transformer = TfidfTransformer()
newspaper_speaker_tfidf = transformer.fit_transform(newspaper_speaker_frequency_csr)

### Save TF-IDF matrix

In [39]:
sparse.save_npz(FILE_NEWSPAPER_SPEAKER_TFIDF, newspaper_speaker_tfidf)